NLP - Aula 4 - Marcação Textual
===============

Nesta aula prática iremos testar as bibliotecas disponiveis em Python para realizar as principais tarefas de marcação textual.


## Bibilotecas POS_TAG, MAC_MORPHO, ##

Vamos testar algumas dessas bibliotecas e observar como elas taguearam as palavras para o português.


### NLTK.POS_TAG

O POS-tagger, processa uma sequência de palavras e adiciona em cada palavra qual parte do discurso ela pertence. Testem a função *nltk.pos_tag(text)*


In [8]:
import nltk
nltk.download("mac_morpho")
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package mac_morpho to
[nltk_data]     D:\Users\542229\AppData\Roaming\nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     D:\Users\542229\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [62]:
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from nltk.corpus import machado 

raw_machado = machado.raw('romance/marm08.txt')

raw_shakespeare = gutenberg.raw('shakespeare-hamlet.txt')
token_shakespeare = word_tokenize(raw_shakespeare)
tag_shakespeare = nltk.pos_tag(token_shakespeare)

tag_shakespeare[:50]

[('[', 'IN'),
 ('The', 'DT'),
 ('Tragedie', 'NNP'),
 ('of', 'IN'),
 ('Hamlet', 'NNP'),
 ('by', 'IN'),
 ('William', 'NNP'),
 ('Shakespeare', 'NNP'),
 ('1599', 'CD'),
 (']', 'NNP'),
 ('Actus', 'NNP'),
 ('Primus', 'NNP'),
 ('.', '.'),
 ('Scoena', 'NNP'),
 ('Prima', 'NNP'),
 ('.', '.'),
 ('Enter', 'NNP'),
 ('Barnardo', 'NNP'),
 ('and', 'CC'),
 ('Francisco', 'NNP'),
 ('two', 'CD'),
 ('Centinels', 'NNP'),
 ('.', '.'),
 ('Barnardo', 'NNP'),
 ('.', '.'),
 ('Who', 'WP'),
 ("'s", 'VBZ'),
 ('there', 'RB'),
 ('?', '.'),
 ('Fran', 'NNP'),
 ('.', '.'),
 ('Nay', 'NNP'),
 ('answer', 'IN'),
 ('me', 'PRP'),
 (':', ':'),
 ('Stand', 'NNP'),
 ('&', 'CC'),
 ('vnfold', 'VB'),
 ('your', 'PRP$'),
 ('selfe', 'JJ'),
 ('Bar', 'NNP'),
 ('.', '.'),
 ('Long', 'NNP'),
 ('liue', 'VBD'),
 ('the', 'DT'),
 ('King', 'NNP'),
 ('Fran', 'NNP'),
 ('.', '.'),
 ('Barnardo', 'NNP'),
 ('?', '.')]

### MAC_MORPHO###

Vamos testar a biblioteca Mac-Mopho, suas palavras e sentenças tagueadas: *nltk.corpus.mac_morpho.tagged_words()* e *nltk.corpus.mac_morpho.tagged_sents ()*

In [28]:
nltk.corpus.mac_morpho.tagged_sents() [:50]


[[('Jersei', 'N'),
  ('atinge', 'V'),
  ('média', 'N'),
  ('de', 'PREP'),
  ('Cr$', 'CUR'),
  ('1,4', 'NUM'),
  ('milhão', 'N'),
  ('em', 'PREP|+'),
  ('a', 'ART'),
  ('venda', 'N'),
  ('de', 'PREP|+'),
  ('a', 'ART'),
  ('Pinhal', 'NPROP'),
  ('em', 'PREP'),
  ('São', 'NPROP'),
  ('Paulo', 'NPROP')],
 [('Programe', 'V'),
  ('sua', 'PROADJ'),
  ('viagem', 'N'),
  ('a', 'PREP|+'),
  ('a', 'ART'),
  ('Exposição', 'NPROP'),
  ('Nacional', 'NPROP'),
  ('do', 'NPROP'),
  ('Zebu', 'NPROP'),
  (',', ','),
  ('que', 'PRO-KS-REL'),
  ('começa', 'V'),
  ('dia', 'N'),
  ('25', 'N|AP')],
 [('Safra', 'N'),
  ('recorde', 'ADJ'),
  ('e', 'KC'),
  ('disponibilidade', 'N'),
  ('de', 'PREP'),
  ('crédito', 'N'),
  ('ativam', 'V'),
  ('vendas', 'N'),
  ('de', 'PREP'),
  ('máquinas', 'N'),
  ('agrícolas', 'ADJ')],
 [('A', 'ART'),
  ('degradação', 'N'),
  ('de', 'PREP|+'),
  ('as', 'ART'),
  ('terras', 'N'),
  ('por', 'PREP|+'),
  ('o', 'ART'),
  ('mau', 'ADJ'),
  ('uso', 'N'),
  ('de', 'PREP|+'),
  ('os',

## Modelos n-gram ##


O diretor de marketing de mecanismos de busca da *DataSciencester* deseja criar milhares de páginas da Web sobre ciência de dados para que seu site tenha uma classificação mais alta nos resultados de pesquisa para termos relacionados à ciência de dados. (Você tenta explicar a ela que os algoritmos dos mecanismos de pesquisa são inteligentes o suficiente para que isso não funcione, mas ele se recusa a ouvir.)

Claro, ele não quer escrever milhares de páginas da web, nem quer pagar uma horda de "estrategistas de conteúdo" para fazê-lo. Em vez disso, ele pergunta se você pode de alguma forma gerar programaticamente essas páginas da web. Para fazer isso, precisaremos de um jeito de modelar a linguagem.

Uma abordagem é começar com um corpus de documentos e aprender um modelo estatístico de linguagem. No nosso caso, começaremos com o ensaio de Mike Loukides, ["O que é ciência de dados?"](https://www.oreilly.com/ideas/what-is-data-science)

Usaremos solicitações e BeautifulSoup para recuperar os dados. Há um par de questões que merecem atenção.

A primeira é que os apóstrofos no texto são, na verdade, o caractere Unicode `u"\u2019"`. Vamos criar uma função auxiliar para substituí-los por apóstrofos normais:

In [30]:
from bs4 import BeautifulSoup
import requests
import re
import random
from collections import defaultdict

url = "https://www.oreilly.com/ideas/what-is-data-science"

def fix_unicode(text):
    return text.replace(u"\u2019","'")
    

In [39]:
def get_document(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html,'html5lib')
    
    document = ' '
    
    for p in soup.find_all('p'):
        document += p.get_text()
        
    for li in soup.find_all('li'):
        document += li.get_text()
    
    return document

A segunda questão é que, assim que recebermos o texto da página da web, queremos dividi-la em uma sequência de palavras e pontos (para que possamos saber onde as frases terminam). Podemos fazer isso usando re.findall():

In [63]:
#document = get_document(url)
document = raw_machado
token_document = word_tokenize(document)

Nós certamente poderíamos (e provavelmente deveríamos) limpar esses dados ainda mais. Ainda há alguma quantidade de texto estranho no documento (por exemplo, a primeira palavra é "We've") e dividimos as frases a partir de pontos que não são, na verdade, pontos finais (por exemplo, em "Web 2.0"), e há um punhado de legendas e listas por toda parte. Dito isso, trabalharemos com o documento como ele é.

Agora que temos o texto como uma sequência de palavras, podemos modelar a linguagem da seguinte maneira: dada uma palavra inicial (digamos, "web"), olhamos todas as palavras que a seguem nos documentos de origem (aqui "2", "is", "front", "was", "server", "has", "friend", "services" etc). Nós escolhemos aleatoriamente uma dessas para ser a próxima palavra, e repetimos o processo até chegarmos a um ponto, que significa o fim da frase. Chamamos isso de *modelo bigrama*, como é determinado completamente pelas freqüências dos bigramas (pares de palavras) nos dados originais.

Que tal uma palavra inicial? Podemos escolher aleatoriamente de palavras que seguem um ponto. Para começar, vamos precomputar as possíveis transições de palavras. Lembre-se de que o `zip` para quando qualquer uma de suas entradas terminar, de modo que `zip(document, document[1:])` fornece precisamente os pares de elementos consecutivos do documento:

In [64]:
bigrams = zip(token_document, token_document[1:])
transitions = defaultdict(list)

for prev,current in bigrams:
    transitions[prev].append(current)
    
transitions

defaultdict(list,
            {'Romance': [','],
             ',': ['Dom',
              '1899',
              'vol',
              'Nova',
              'Rio',
              '1994',
              'Rio',
              '1899',
              'vindo',
              'encontrei',
              'que',
              'sentou-se',
              'falou',
              'e',
              'e',
              'porém',
              'que',
              'como',
              'fechei',
              'disse',
              'murmurou',
              'mas',
              'e',
              'que',
              'deram',
              'que',
              'e',
              'por',
              'chamam-me',
              'alguns',
              'domingo',
              'Dom',
              'e',
              'não',
              'dou-lhe',
              'dou-lhe',
              'mas',
              'para',
              'vai',
              'sendo',
              'poderá',
              'passo',
          

Agora estamos prontos para gerar sentenças:

In [65]:
def generate_using_bigrams(transitions):
    current = "."
    result = []
    
    while True:
        next_candidates = transitions[current]
        current = random.choice(next_candidates)
        result.append(current)
        
        if current == "." : return " ".join(result)
    

As frases produzidas são sem sentido, mas são o tipo de tagarelice que você pode colocar no seu site se estiver tentando soar como se fosse um site de ciência dos dados. Por exemplo:

In [69]:
generate_using_bigrams(transitions)

'Um dos bons livros e Capitu , como a falar assim como ela : \x97 Pois um tanto que os meus ficaram diante e , falar-lhe-ia à porta da véspera e quis aprender em nossa separação é aqui , enfiado , pensei em tom que se a tal vergonha ! Não voltavam a Capitu brincava de nascer com o resto .'

Podemos tornar as frases menos sem sentido, olhando para trigramas, triplas de palavras consecutivas. (Em geral, você pode ver n-gramas consistindo de n palavras consecutivas, mas três serão suficientes para nós.) Agora as transições dependerão das duas palavras anteriores:

In [115]:
trigrams = zip(token_document, token_document[1:], token_document[2:])
trigram_transitions = defaultdict(list)
starts = []

for prev,current, next in trigrams:
    if prev == '.':
        starts.append(current)
    trigram_transitions[(prev,current)].append(next)
    
starts

['I',
 'Publicado',
 'CAPÍTULO',
 'Cumprimentou-me',
 'A',
 'Sucedeu',
 '\x97',
 '\x97',
 '\x97',
 'Vi-lhe',
 'No',
 'Os',
 'Nem',
 'Contei',
 'Não',
 'Casmurro',
 'Dom',
 'Tudo',
 'O',
 'E',
 'Há',
 'CAPÍTULO',
 'Antes',
 'Vivo',
 'A',
 'Um',
 'Construtor',
 'Na',
 'Nos',
 'Quando',
 'Naturalmente',
 'O',
 'Tenho',
 'Uso',
 'Enfim',
 'O',
 'Pois',
 'Em',
 'Se',
 'O',
 'Uma',
 'Os',
 'Quanto',
 'Duas',
 'Entretanto',
 'A',
 'Em',
 'Distrações',
 'O',
 'Ora',
 'Quis',
 'Jurisprudência',
 'Depois',
 'Foi',
 'Talvez',
 'Sim',
 'Deste',
 'Eia',
 'Tive',
 'É',
 'CAPÍTULO',
 'A',
 '\x97',
 '\x97',
 'Minha',
 'José',
 '\x97',
 'Não',
 '\x97',
 'Metidos',
 'Em',
 'Bentinho',
 'A',
 'Basta',
 'Capitu',
 'Não',
 'Pois',
 '\x97',
 'Oxalá',
 '\x97',
 'Bentinho',
 'E',
 'Não',
 '\x97',
 'O',
 '\x97',
 'Quanto',
 'Mas',
 '\x97',
 'Contudo',
 'Prima',
 'Seguiu-se',
 'Prima',
 'Era',
 'Levantou-se',
 'Cosi-me',
 'Foi',
 'Trazia',
 'A',
 'O',
 'Era',
 'Levantou-se',
 'Um',
 'Também',
 'Outrossim',
 'No

Observe que agora temos que rastrear as palavras iniciais separadamente. Podemos gerar sentenças praticamente da mesma maneira:

In [76]:
def generate_using_trigrams(starts, trigram_transitions):
    current = random.choice(starts)
    prev = "."
    result = [current]
    
    while True:
        next_candidates = trigram_transitions[(prev,current)]
        next = random.choice(next_candidates)
        prev = current
        current = next
        
        result.append(current)
        
        if current == "." : return " ".join(result)

Essa função vai produzir sentenças como:

In [114]:
generate_using_trigrams(starts, trigram_transitions)

'Vi as últimas palavras , mas ia falando do pai do morto .'

Claro, elas soam melhor porque a cada passo o processo de geração tem menos escolhas, e em muitos passos há apenas uma única escolha. Isso significa que você freqüentemente gerará frases (ou pelo menos frases longas) que foram vistas literalmente nos dados originais. Ter mais dados ajudaria; também funcionaria melhor se você coletasse *n-gramas* de vários ensaios sobre ciência de dados.